In [1]:
"""
Recognition demo 
"""
%matplotlib inline
import random
import time
from pathlib import Path

import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from recognition_model import Recognition_model
from utils import preprocess_image, process_bboxes, preprocess_label, process_tp
from visualization import plt_bboxes, bboxes_draw_on_img
from read_txt import Read_txt, load_dataset, load_npz, save_dataset2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
LABELS=["頭屋 造橋 右線",
"國道三號",
"頭份",
"頭份 14 國道三號 25",
"頭份 6 國道三號 17",
"旅行時間標誌板 頭份 新竹 中壢",
"頭份 三灣 右線",
"新竹",
"國道三號 9 新竹 13",
"竹南 竹東 2000m",
"台北 竹東 右線",
"竹北",
"竹南",
"後龍 竹南 右線",
"新竹科學園區 右線",
"湖口",
"新竹 竹東 右線",
"楊梅",
"竹北 芎林 右線",
"中壢 林口 約 分",
"新豐 湖口 新竹工業區 右線",
"中壢",
"楊梅 12 中壢 19",
"楊梅 中壢 2500m",
"66快速道路",
"中壢 桃園機場",
"楊梅 埔心 右線",
]

LABELS=["非路牌",
"其他",
"頭屋 造橋 右線",
"國道三號",
"頭份",
"頭屋 造橋 箭頭右",
"頭份 14 國道三號 25",
"頭份 6 國道三號 17",
"旅行時間標誌板 頭份 新竹 中壢",
"頭份 三灣 出口2公里",
"頭份 三灣 右線",
"新竹",
"頭份 三灣 箭頭右",
"國道三號 9 新竹 13",
"竹南 竹東 2000m",
"竹南 竹東 1500m",
"台北 竹東 右線",
"竹北",
"竹南",
"台北 竹東 箭頭右",
"後龍 竹南 右線",
"後龍 竹南 箭頭右",
"新竹科學園區 右線",
"湖口",
"新竹 竹東 出口1.5公里",
"新竹科學園區 箭頭右",
"新竹 竹東 右線",
"楊梅",
"新竹 竹東 箭頭右",
"竹北 芎林 右線",
"竹北 芎林 箭頭右",
"中壢 林口 約 分",
"新豐 湖口 新竹工業區 出口2公里",
"新豐 湖口 新竹工業區 右線",
"中壢",
"新豐 湖口 新竹工業區 箭頭右",
"楊梅 12 中壢 19",
"楊梅 中壢 2500m",
"楊梅 中壢 1000m",
"66快速道路",
"中壢 桃園機場",
"楊梅 埔心 右線",
]

# Read text

In [3]:
def read(txt_name, dir_path):
    with open(txt_name,"r") as fp:
        all_lines = fp.readlines()

    text = []
    
    for line in all_lines:
        line = line[:-1].split(",")
        name = line[0]

        cla =int(line[1])

        text.append({"name": str(dir_path/name), "class": [cla]})

    return text

In [4]:
txt_name = Path.cwd().parent / "ssd_data_panel" / "ssd_data_panel_cls.txt" # 1753
dir_path = Path.cwd().parent / "ssd_data_panel"

text1 = read(str(txt_name), dir_path)
print(len(text1))

txt_name = Path.cwd().parent / "day_rain_frames_panel" / "day_rain_frames_panel_cls.txt" # 2767
dir_path = Path.cwd().parent / "day_rain_frames_panel"

text2 = read(str(txt_name), dir_path)
print(len(text2))

txt_name = Path.cwd().parent / "night_frames_panel" / "night_frames_panel_cls.txt" # 2815
dir_path = Path.cwd().parent / "night_frames_panel"

text3 = read(str(txt_name), dir_path)
print(len(text3))

txt_name = Path.cwd().parent / "day_frames_update_panel" / "day_frames_update_gt.txt" # 4612
dir_path = Path.cwd().parent / "day_frames_update_panel"

text4 = read(str(txt_name), dir_path)
print(len(text4))

1753
2767
2815
4612


In [7]:
text =  text1 + text2 + text3 + text4 # text1 text2 +

label_dict = {}
train_list = []
test_list = []
RATIO = 0.7

#train_list = text
#test_list = text4

for data in text:
    if str(data["class"][0]) not in label_dict:
        label_dict[str(data["class"][0])] = []
        
    label_dict[str(data["class"][0])].append(data)

    
for key, data in (label_dict.items()):
    random.shuffle(data)
    train_list = train_list + data[:int(len(data)*RATIO)]
    test_list = test_list + data[int(len(data)*RATIO):]

print(len(train_list))
print(len(test_list))

8343
3604


for data in test_list:
    if str(data["class"][0]) not in label_dict:
        label_dict[str(data["class"][0])] = []
        
    label_dict[str(data["class"][0])].append(data)

for key, data in (label_dict.items()):
    print(key, " ", len(data))

sum_w = 0.0
sum_h = 0.0
max_w = 0
max_h = 0

for data in text:
    img = cv2.imread(data["name"])
    if img is None:
        continue
    print("load: ", data["name"])
    
    sum_h += img.shape[0]
    sum_w += img.shape[1]
    
    max_h = np.maximum(img.shape[0], max_h)
    max_w = np.maximum(img.shape[1], max_w)
    
print("avg h=", sum_h / len(text), "avg w=",sum_w / len(text))
print("max_h=", max_h, "max_w=", max_w)


# Predict

In [31]:
from tensorflow.contrib import predictor

model_path = "recognition_model_mobilev2_width0125"

model_name = "saved_model_"+model_path

def get_pred_fn(dir_path):
    export_dir = dir_path
    subdirs = [x for x in Path(export_dir).iterdir()
               if x.is_dir() and 'temp' not in str(x)]
    latest = str(sorted(subdirs)[-1]) # saved_model/1556852589
    return predictor.from_saved_model(latest)

predict_fn = get_pred_fn(model_name)

INFO:tensorflow:Restoring parameters from b'saved_model_recognition_model_mobilev2_width0125\\1563296957\\variables\\variables'


In [32]:
def label_correct(labels):
    correct_list=[0,0,0,1,2,0,3,4,5,6,6,7,
                  6,8,9,9,10,11,12,10,13,13,
                  14,15,16,14,16,17,16,18,18,19,
                  20,20,21,20,22,23,23,24,25,26]
    
    labels[0] = correct_list[labels[0]]
    return labels

In [33]:
def predi(data, tp):
    
    t1 = time.time()    
    
    pred = predict_fn({'x': data["img_prepocessed"]})
    predict_class = np.argmax(pred["tfoutput"][0])
    
    if data["class"][0] == predict_class:
        tp += 1
    else:
        #if pred["tfoutput"][0][predict_class] < 0.5:
        #    return tp
        print("label=", LABELS[data["class"][0]])    
        print("predict=", LABELS[predict_class], pred["tfoutput"][0][predict_class])
        #fig = plt.figure(figsize=(20,20))
        #plt.subplot(1,1,1)
        #plt.imshow(data["img_prepocessed"][0].astype(np.int))
        #plt.show()

    t2 = time.time()
    #print("Cost time = ", t2 - t1)
    
    return tp
    
    #cv2.imwrite(str(Path.cwd() / "output_img")+"/"+str(i).zfill(4)+".jpg", img)

In [34]:
tar = test_list # test_list[:]
print(len(tar))
#random.shuffle(tar)

t1 = time.time()

tp = 0

for idx, target in enumerate(tar):
    img = cv2.imread(target["name"])
    if img is None:
        continue
    #print(idx, "load: ", target["name"])

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    labels = target['class']
    labels = label_correct(labels)

    img_prepocessed = preprocess_image(img_rgb,
                                       out_shape=(128,128),
                                       num=1)

    label_prepocessed = np.asarray(labels, np.int64)
    if label_prepocessed.size == 0:
        continue

    #fig = plt.figure(figsize=(20,20))
    #plt.subplot(1,1,1)
    #plt.imshow(img_prepocessed[0].astype(np.int)) # 顯示圖片
    #plt.subplot(1,2,2)
    #plt.imshow(cropped_image_with_box.astype(np.int)) # 顯示圖片
    #plt.show()

    data = {"img_prepocessed": img_prepocessed, "class": label_prepocessed}
    tp = predi(data, tp)

t2 = time.time()
print("\nCost time = ", t2 - t1)

print("tp=", tp, "len=", len(tar), "accuracy=", tp/len(tar)) # 3565 3293

3604
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.9967997
label= 頭屋 造橋 右線
predict= 頭份 14 國道三號 25 0.839049
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.5970163
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.9698388
label= 頭屋 造橋 右線
predict= 新竹 0.48575428
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.33001575
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.9371048
label= 頭屋 造橋 右線
predict= 新竹 0.6362113
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.9445931
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.95050836
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.53315985
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.71079224
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.9880761
label= 頭屋 造橋 右線
predict= 新竹 0.3827518
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.98273575
label= 頭屋 造橋 右線
predict= 新竹 0.4585326
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.9629438
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.4408421
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.35250738
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.96376705
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.9681603
label= 頭屋 造橋 右線
pre

label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.4719191
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.9935921
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.9986571
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.99930155
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.5241303
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.8069208
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.9978612
label= 頭屋 造橋 右線
predict= 頭份 三灣 右線 0.60758346
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.46784836
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.66715974
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.99886334
label= 頭屋 造橋 右線
predict= 頭份 三灣 右線 0.8132282
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.6026655
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.9988978
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.75920856
label= 頭屋 造橋 右線
predict= 頭份 三灣 右線 0.94161385
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.9996556
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.7874236
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.8274473
label= 頭屋 造橋 右線
predict= 頭份 三灣 右線 0.6140824
label= 頭屋 造橋 右線
predict= 頭份 三灣 右線 0.98304546
label= 頭屋 造橋 右線
predict= 頭份 三灣 右線 0.85161346
label= 頭屋 造

label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.7840509
label= 頭屋 造橋 右線
predict= 國道三號 0.4795914
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.5278706
label= 頭屋 造橋 右線
predict= 頭份 三灣 右線 0.8478331
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.8263011
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.8815679
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.9417105
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.53629965
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.8561259
label= 頭屋 造橋 右線
predict= 國道三號 0.6933499
label= 頭屋 造橋 右線
predict= 國道三號 9 新竹 13 0.999376
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.33494842
label= 頭屋 造橋 右線
predict= 頭份 14 國道三號 25 0.6627637
label= 頭屋 造橋 右線
predict= 頭份 14 國道三號 25 0.34983534
label= 頭屋 造橋 右線
predict= 頭份 14 國道三號 25 0.72519964
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.579331
label= 頭屋 造橋 右線
predict= 台北 竹東 右線 0.18662946
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.70516956
label= 頭屋 造橋 右線
predict= 頭份 6 國道三號 17 0.47565123
label= 頭屋 造橋 右線
predict= 頭份 三灣 右線 0.94022423
label= 頭屋 造橋 右線
predict= 頭份 14 國道三號 25 0.9268633
label= 頭屋 造橋 右

# Train

In [26]:
Batch_SIZE = 32
TRAIN_STEP = 10000
model_path = "./recognition_model_mobilev2_width0125"
OUTPUT_SHAPE = (128,128)

def train(handle, img_list, class_list, steps):    
    data = {"img": np.concatenate(img_list), 
            "class": np.concatenate(class_list),
            "batch": Batch_SIZE}

    print(data["img"].shape)
    
    handle.train(data=data, steps=steps)

In [27]:
def label_correct(labels):
    correct_list=[0,0,0,1,2,0,3,4,5,6,6,7,
                  6,8,9,9,10,11,12,10,13,13,
                  14,15,16,14,16,17,16,18,18,19,
                  20,20,21,20,22,23,23,24,25,26]
    
    labels[0] = correct_list[labels[0]]
    return labels

In [28]:
handle = Recognition_model(model_path = model_path)

tar = train_list[:]
random.shuffle(tar)

counter = 0
img_list = []
class_list = []
for idx, target in enumerate(tar):
    img = cv2.imread(target["name"])
    if img is None:
        continue
    print(idx, "load: ", target["name"])

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    labels = target['class']
    
    if (labels[0] == 0 or labels[0] == 1):
        print(labels[0],'---------------------------')
        continue
    labels = label_correct(labels)

    img_prepocessed = preprocess_image(img_rgb,
                                       out_shape=OUTPUT_SHAPE,
                                       num=1)

    label_prepocessed = np.asarray(labels, np.int64)
    if label_prepocessed.size == 0:
        continue

    #break
    #fig = plt.figure(figsize=(20,20))
    #plt.subplot(1,1,1)
    #plt.imshow(img_prepocessed[0].astype(np.int)) # 顯示圖片
    #plt.subplot(1,2,2)
    #plt.imshow(cropped_image_with_box.astype(np.int)) # 顯示圖片
    #plt.show()

    # Batch
    img_list.append(img_prepocessed)
    class_list.append(label_prepocessed)
    counter += 1

    #print("idx+1=",idx+1)
    #if (idx+1) % Batch_SIZE == 0:
    #    train(handle, img_list, class_list, steps=TRAIN_STEP)
    #    img_list = []
    #    class_list = []
print(counter)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './recognition_model_mobilev2_width0125', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D9C1A80C50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
{'layer1': <tf.Tensor 'conv1_1/Relu6:0' shape=(1, 64, 64, 32) dtype=float32>, 'layer2': <tf.Tensor 'res2_1/res2_1project_conv2d/res2_1project_conv2db_norm/FusedBatchNorm:0' shape=(1, 64, 64, 2) dtype=float32>, 'layer3': <tf.Tensor 'res3_2/add:0' shape=(1, 32, 32, 3) dtype=float32

217 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\001555_3.jpg
218 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\003543_2.jpg
219 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\07796_0.jpg
220 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003326_0.jpg
221 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003161_1.jpg
222 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\001613_0.jpg
223 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\001677_0.jpg
224 load:  C:\file\M0659416\SVNRepository\night_frames_panel\001304_0.jpg
225 load:  C:\file\M0659416\SVNRepository\night_frames_panel\000403_0.jpg
0 ---------------------------
226 load:  C:\file\M0659416\SVNRepository\night_frames_panel\000884_0.jpg
227 load:  C:\file\M0659416\SVNRepository\night_frames_panel\000542_0.jpg
228 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000332_0.jpg
229 load:  C:\file\M0659416\SVNRepository\day_frames_upd

447 load:  C:\file\M0659416\SVNRepository\night_frames_panel\002543_2.jpg
448 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\003586_1.jpg
449 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000961_1.jpg
1 ---------------------------
450 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\04291_2.jpg
451 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\00726_1.jpg
0 ---------------------------
452 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\003887_0.jpg
453 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000892_1.jpg
1 ---------------------------
454 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\07516_0.jpg
455 load:  C:\file\M0659416\SVNRepository\night_frames_panel\000913_0.jpg
456 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003191_1.jpg
457 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000931_2.jpg
458 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000890_0.jpg
459 lo

671 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\002052_3.jpg
672 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003862_2.jpg
673 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\003580_1.jpg
674 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000136_1.jpg
0 ---------------------------
675 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\00456_0.jpg
0 ---------------------------
676 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\002470_3.jpg
677 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\001353_0.jpg
678 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003351_0.jpg
679 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000132_0.jpg
1 ---------------------------
680 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\003995_0.jpg
681 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\003671_0.jpg
682 load:  C:\file\M0659416\SVNRepository\day_frames_update

889 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\002408_3.jpg
890 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\002437_2.jpg
891 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\001062_0.jpg
892 load:  C:\file\M0659416\SVNRepository\night_frames_panel\002434_0.jpg
893 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\04522_0.jpg
894 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000682_0.jpg
895 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\001169_0.jpg
896 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\02134_1.jpg
1 ---------------------------
897 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\004068_0.jpg
898 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\001278_0.jpg
899 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\000290_0.jpg
1 ---------------------------
900 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\000431_0.jpg
1 ---------------------

...

7502 load:  C:\file\M0659416\SVNRepository\night_frames_panel\001333_1.jpg
7503 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\04899_3.jpg
7504 load:  C:\file\M0659416\SVNRepository\night_frames_panel\000882_3.jpg
7505 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003205_0.jpg
7506 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\001566_0.jpg
7507 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\001672_0.jpg
7508 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\02141_2.jpg
7509 load:  C:\file\M0659416\SVNRepository\night_frames_panel\001331_0.jpg
7510 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\000627_0.jpg
0 ---------------------------
7511 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\002077_3.jpg
7512 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\06035_0.jpg
7513 load:  C:\file\M0659416\SVNRepository\night_frames_panel\002590_2.jpg
7514 load:  C:\file\M0659416\SVNRepository\night_frames_panel\00

7720 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\03030_0.jpg
7721 load:  C:\file\M0659416\SVNRepository\night_frames_panel\002579_2.jpg
7722 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\06570_0.jpg
7723 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003732_0.jpg
7724 load:  C:\file\M0659416\SVNRepository\night_frames_panel\001843_0.jpg
7725 load:  C:\file\M0659416\SVNRepository\night_frames_panel\001097_0.jpg
7726 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000958_0.jpg
7727 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000027_1.jpg
0 ---------------------------
7728 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003321_0.jpg
7729 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\002037_0.jpg
7730 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\00742_2.jpg
0 ---------------------------
7731 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\04264_2.jpg
7732 load:  C:\file\M0659416\SVNRep

7918 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\000135_0.jpg
0 ---------------------------
7919 load:  C:\file\M0659416\SVNRepository\night_frames_panel\001310_3.jpg
7920 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\04866_1.jpg
7921 load:  C:\file\M0659416\SVNRepository\ssd_data_panel\07754_1.jpg
7922 load:  C:\file\M0659416\SVNRepository\night_frames_panel\000874_3.jpg
7923 load:  C:\file\M0659416\SVNRepository\night_frames_panel\002083_0.jpg
7924 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000520_0.jpg
0 ---------------------------
7925 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\002692_0.jpg
7926 load:  C:\file\M0659416\SVNRepository\night_frames_panel\002553_3.jpg
7927 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\000632_0.jpg
0 ---------------------------
7928 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003131_0.jpg
7929 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\000272_2.

8116 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000082_2.jpg
0 ---------------------------
8117 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\003897_0.jpg
8118 load:  C:\file\M0659416\SVNRepository\night_frames_panel\000725_0.jpg
8119 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003866_2.jpg
8120 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\001806_0.jpg
8121 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003227_1.jpg
8122 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\003600_0.jpg
8123 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\002452_2.jpg
8124 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\003679_0.jpg
8125 load:  C:\file\M0659416\SVNRepository\day_rain_frames_panel\001620_1.jpg
8126 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\004090_3.jpg
8127 load:  C:\file\M0659416\SVNRepository\night_frames_panel\001920_0.jpg
8128 load:  C:\file\M0659416

0 ---------------------------
8341 load:  C:\file\M0659416\SVNRepository\day_frames_update_panel\000888_2.jpg
8342 load:  C:\file\M0659416\SVNRepository\night_frames_panel\002082_0.jpg
6928


for i, img in enumerate(img_list[:50]):
    #fig = plt.figure(figsize=(20,20))
    print(LABELS[class_list[i][0]])
    
    plt.subplot(1,1,1)
    plt.imshow(img[0].astype(np.int)) # 顯示圖片
    plt.show()
    
    

In [29]:
t1 = time.time()
    
train(handle, img_list, class_list, steps=TRAIN_STEP)

t2 = time.time()
print("\nCost time = ", t2 - t1)

(6928, 128, 128, 3)
----Start train----
INFO:tensorflow:Calling model_fn.
{'layer1': <tf.Tensor 'conv1_1/Relu6:0' shape=(32, 64, 64, 32) dtype=float32>, 'layer2': <tf.Tensor 'res2_1/res2_1project_conv2d/res2_1project_conv2db_norm/FusedBatchNorm:0' shape=(32, 64, 64, 2) dtype=float32>, 'layer3': <tf.Tensor 'res3_2/add:0' shape=(32, 32, 32, 3) dtype=float32>, 'layer4': <tf.Tensor 'res4_3/add:0' shape=(32, 16, 16, 4) dtype=float32>, 'layer5': <tf.Tensor 'res5_4/add:0' shape=(32, 8, 8, 8) dtype=float32>, 'layer6': <tf.Tensor 'res6_3/add:0' shape=(32, 8, 8, 12) dtype=float32>, 'layer7': <tf.Tensor 'res7_3/add:0' shape=(32, 4, 4, 20) dtype=float32>, 'layer8': <tf.Tensor 'res8_1/res8_1project_conv2d/res8_1project_conv2db_norm/FusedBatchNorm:0' shape=(32, 4, 4, 40) dtype=float32>, 'layer9': <tf.Tensor 'layer9_1/Relu6:0' shape=(32, 4, 4, 512) dtype=float32>, 'layer10': <tf.Tensor 'avg_10_1/AvgPool:0' shape=(32, 1, 1, 512) dtype=float32>, 'layer11': <tf.Tensor 'squeeze11_2:0' shape=(32, 27) dtyp

INFO:tensorflow:loss = 11.48027, step = 20000
INFO:tensorflow:global_step/sec: 19.6525
INFO:tensorflow:probabilities = [[0.90682524 0.00026632 0.00635453 0.00025462 0.00013015 0.00067419
  0.01490261 0.00004407 0.00082488 0.04712895 0.01347957 0.00002418
  0.00003519 0.00376704 0.00007754 0.00001507 0.0032302  0.00003133
  0.00006162 0.00038652 0.00011937 0.00024406 0.00017111 0.00001382
  0.00002857 0.0001687  0.00074025]
 [0.0000642  0.0000235  0.00003678 0.00132399 0.00002655 0.00000161
  0.00647712 0.00001742 0.0000456  0.96603405 0.01849591 0.00020961
  0.00003487 0.00045066 0.00413225 0.0000178  0.0000492  0.00008756
  0.00181882 0.00021024 0.00002132 0.0000544  0.00006229 0.00003636
  0.00002882 0.00022364 0.00001548]
 [0.00367334 0.01203791 0.00702355 0.1022153  0.07936934 0.00662644
  0.07744911 0.00615583 0.61990815 0.00726427 0.0049029  0.00733011
  0.02462772 0.01047529 0.00172793 0.00355649 0.00326569 0.00453289
  0.00093565 0.00210157 0.00096527 0.00227554 0.00440926 0.00

INFO:tensorflow:loss = 0.22936338, step = 20100 (5.091 sec)
INFO:tensorflow:global_step/sec: 24.0276
INFO:tensorflow:probabilities = [[0.00000386 0.00014446 0.00009361 0.9959985  0.00010241 0.00004129
  0.00080836 0.00028073 0.00022333 0.00041751 0.00024903 0.00010614
  0.00021366 0.00001033 0.00022115 0.00009254 0.00012639 0.00008634
  0.00012279 0.00002058 0.00000078 0.00001641 0.00049034 0.00011211
  0.00000945 0.00000212 0.00000585]
 [0.00000113 0.00000039 0.00010404 0.00000348 0.00007708 0.00000594
  0.00002485 0.99968517 0.00000148 0.00000418 0.00000044 0.00003374
  0.00000107 0.00000254 0.000004   0.00001662 0.00000263 0.00000076
  0.00000012 0.00000047 0.00000015 0.00000129 0.00000066 0.00000089
  0.00002592 0.00000034 0.00000066]
 [0.00072141 0.00009145 0.98119557 0.00024518 0.00026513 0.00007886
  0.00749322 0.00424308 0.00009998 0.00336379 0.00003415 0.00014676
  0.00022042 0.00042047 0.00035227 0.00005785 0.00003233 0.00009795
  0.00002208 0.00006397 0.00000946 0.00042155 0

INFO:tensorflow:loss = 0.036070444, step = 20200 (4.162 sec)
INFO:tensorflow:global_step/sec: 24.1492
INFO:tensorflow:probabilities = [[0.00000344 0.00001435 0.00000687 0.00003636 0.00075284 0.00043939
  0.0000189  0.00000262 0.99854624 0.00000737 0.00001938 0.00000735
  0.00002255 0.00003681 0.00000088 0.00000195 0.00000339 0.00000165
  0.00000009 0.00000119 0.00000407 0.00000539 0.00001386 0.00001707
  0.00002133 0.00000709 0.0000079 ]
 [0.00000021 0.00000012 0.00000011 0.00000056 0.00008301 0.00005245
  0.00000093 0.00000002 0.9997987  0.00000059 0.00000349 0.0000105
  0.00000173 0.00001786 0.00000014 0.00000022 0.00000002 0.00000025
  0.00000001 0.00000019 0.00000933 0.00000052 0.00000065 0.00000843
  0.00000115 0.00000848 0.00000033]
 [0.00004946 0.00001301 0.00001464 0.00059957 0.00000958 0.00001008
  0.9982703  0.0000257  0.00012091 0.00000536 0.00010953 0.00002056
  0.00018748 0.00011426 0.00001714 0.00014318 0.00015997 0.0000067
  0.00004428 0.00000413 0.00001158 0.0000362  0.

INFO:tensorflow:loss = 0.04475064, step = 20300 (4.142 sec)
INFO:tensorflow:global_step/sec: 23.913
INFO:tensorflow:probabilities = [[0.00004259 0.0000158  0.00000203 0.0002056  0.00000379 0.00000052
  0.99937975 0.00001455 0.00001736 0.0000027  0.00003648 0.00001567
  0.00012925 0.00003118 0.00000486 0.00002547 0.00003168 0.00000198
  0.00002348 0.00000113 0.00000215 0.00000216 0.00000397 0.0000002
  0.00000229 0.00000087 0.00000233]
 [0.00000005 0.00002037 0.00000491 0.9983991  0.00001272 0.00000029
  0.00142669 0.00004516 0.00000086 0.00000451 0.00000295 0.00000577
  0.00001308 0.00000011 0.0000041  0.00001349 0.00001806 0.00001622
  0.0000073  0.0000003  0.00000002 0.00000039 0.00000308 0.00000064
  0.00000003 0.00000001 0.00000012]
 [0.00000349 0.00000215 0.00017536 0.00002056 0.0000085  0.00000013
  0.00000706 0.00000743 0.00000313 0.9988782  0.00014659 0.00000099
  0.00000125 0.00015449 0.0005165  0.00000049 0.00000535 0.00001211
  0.00000553 0.00001247 0.00000053 0.00000585 0.0

...

INFO:tensorflow:loss = 0.00018633528, step = 29500 (4.149 sec)
INFO:tensorflow:global_step/sec: 23.7207
INFO:tensorflow:probabilities = [[0.9974624  0.00002121 0.0005862  0.00000079 0.00000284 0.00052083
  0.00114731 0.00000739 0.00007709 0.00002486 0.00006235 0.00000315
  0.00000052 0.00000489 0.00000029 0.00000195 0.00000961 0.00000088
  0.0000005  0.00001863 0.00002083 0.00000856 0.00000198 0.00000029
  0.00000086 0.00000475 0.00000937]
 [0.00000001 0.         0.00000001 0.00000109 0.00000002 0.
  0.99999785 0.00000013 0.00000006 0.00000001 0.00000022 0.00000005
  0.00000009 0.00000029 0.00000001 0.00000014 0.00000011 0.
  0.00000006 0.         0.00000001 0.00000003 0.00000001 0.
  0.00000001 0.         0.00000001]
 [0.00000013 0.00000031 0.00000732 0.00000036 0.00000187 0.00000049
  0.0000008  0.99997663 0.00000013 0.00000058 0.00000015 0.00000579
  0.00000003 0.00000002 0.00000087 0.00000167 0.00000009 0.00000042
  0.00000002 0.00000016 0.00000001 0.00000011 0.00000009 0.00000005


INFO:tensorflow:loss = 0.0018317333, step = 29600 (4.216 sec)
INFO:tensorflow:global_step/sec: 24.0736
INFO:tensorflow:probabilities = [[0.00000005 0.9999895  0.00000821 0.00000008 0.00000012 0.00000019
  0.00000013 0.00000041 0.0000002  0.00000036 0.0000002  0.
  0.         0.00000001 0.00000013 0.00000033 0.00000002 0.00000003
  0.         0.00000002 0.00000001 0.         0.         0.
  0.00000001 0.         0.        ]
 [0.00000034 0.00000742 0.00001863 0.9998493  0.00003974 0.00000286
  0.00000983 0.00001501 0.00000386 0.00000986 0.00000199 0.00000468
  0.00001001 0.00000016 0.00000251 0.00000172 0.0000015  0.00000463
  0.00000388 0.00000113 0.00000004 0.00000096 0.00000679 0.00000295
  0.0000001  0.00000003 0.00000025]
 [0.00000002 0.00000004 0.00000003 0.00000014 0.00001412 0.00001715
  0.00000009 0.00000002 0.9999604  0.00000022 0.00000019 0.00000234
  0.00000039 0.0000006  0.00000003 0.00000021 0.         0.00000008
  0.         0.00000002 0.00000024 0.0000002  0.00000014 0.00

INFO:tensorflow:loss = 8.7639084e-05, step = 29700 (4.155 sec)
INFO:tensorflow:global_step/sec: 24.0163
INFO:tensorflow:probabilities = [[0.00000483 0.00000051 0.00000005 0.00000054 0.00000009 0.00000002
  0.99996257 0.00000025 0.00000239 0.00000019 0.00002058 0.0000003
  0.00000092 0.00000362 0.00000008 0.00000032 0.00000102 0.00000004
  0.0000003  0.00000005 0.00000042 0.00000014 0.0000001  0.00000001
  0.00000013 0.00000009 0.00000025]
 [0.0000017  0.00000106 0.00005875 0.0000003  0.00000752 0.00000245
  0.00000105 0.9999021  0.00000101 0.00000071 0.00000013 0.0000063
  0.00000014 0.00000014 0.00000144 0.00000198 0.00000022 0.00000089
  0.00000004 0.00000062 0.00000004 0.00000089 0.00000019 0.00000008
  0.00001004 0.00000027 0.00000008]
 [0.00000009 0.         0.         0.         0.00000002 0.
  0.9999999  0.         0.00000001 0.         0.00000001 0.
  0.         0.00000004 0.         0.         0.00000001 0.
  0.         0.         0.00000001 0.         0.         0.
  0.      

INFO:tensorflow:loss = 0.00035304716, step = 29800 (4.164 sec)
INFO:tensorflow:global_step/sec: 23.7881
INFO:tensorflow:probabilities = [[0.00000049 0.00000023 0.00000005 0.0000051  0.00000037 0.00000006
  0.99998486 0.00000057 0.00000148 0.00000002 0.00000275 0.00000025
  0.00000055 0.00000084 0.00000002 0.00000058 0.00000079 0.00000002
  0.00000018 0.00000003 0.00000037 0.00000011 0.00000006 0.00000001
  0.00000004 0.00000002 0.00000015]
 [0.         0.00000009 0.         0.0000001  0.00000206 0.00000123
  0.00000014 0.         0.9999957  0.00000004 0.00000029 0.00000003
  0.00000003 0.00000012 0.         0.         0.         0.00000001
  0.         0.         0.00000003 0.00000002 0.00000003 0.00000008
  0.00000016 0.00000005 0.00000001]
 [0.00000016 0.00000014 0.9999969  0.00000009 0.00000005 0.00000004
  0.00000098 0.00000017 0.00000004 0.00000069 0.         0.00000002
  0.00000038 0.00000008 0.0000001  0.00000008 0.00000001 0.00000003
  0.         0.00000001 0.         0.0000002

INFO:tensorflow:loss = 0.0005783513, step = 29900 (4.204 sec)
INFO:tensorflow:Saving checkpoints for 30000 into ./recognition_model_mobilev2_width0125\model.ckpt.
INFO:tensorflow:Loss for final step: 0.0003049539.
----End train----

Cost time =  429.5441081523895


# Export savemodel

In [30]:
model_path = "recognition_model_mobilev2_width0125"
handle = Recognition_model(model_path = "./"+model_path) # SSD_model_mobilev1 SSD_model_mobilev2
handle.save_model("saved_model_" + model_path)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './recognition_model_mobilev2_width0125', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D9AD5E7780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
{'layer1': <tf.Tensor 'conv1_1/Relu6:0' shape=(1, 64, 64, 32) dtype=float32>, 'layer2': <tf.Tensor 'res2_1/res2_1project_conv2d/res2_1project_conv2db_norm/FusedBatchNorm:0' shape=(1, 64, 64, 2) dtype=float32>, 'layer3': <tf.Tensor 'res3_2/add:0' shape=(1, 32, 32, 3) dtype=float32